In [1]:
import cv2
import numpy as np
import math
import os
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras
from tensorflow import keras
from IPython.display import clear_output
from IPython.utils import io
import mediapipe as mp

# Approach 1 Demo

#### Body Pose

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

#### Face

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh



# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_tesselation_style())
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_iris_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

#### Hands

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(model_complexity=0,min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    while True:
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image,hand_landmarks,mp_hands.HAND_CONNECTIONS)
                # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
        if cv2.waitKey(20) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

# Final Approach Image Preprocessing

In [ ]:
cap = cv2.VideoCapture(0) 
while True:
    ret, frame = cap.read() 
    
    mirror = cv2.flip(frame, 1) 
    
    fh, fw = mirror.shape[:2] 
    
    rois = int(fw/3) 
    
    sq1 = mirror[0:rois, 0:rois] 
    
    grey = cv2.cvtColor(sq1, cv2.COLOR_BGR2GRAY) 
    
    grey = np.repeat(grey[...,np.newaxis], 3, -1)
    
    sq2 = mirror[0:rois, rois:2*rois]
    
    blurred = cv2.cvtColor(sq2, cv2.COLOR_BGR2GRAY)
    
    
    blurred = cv2.GaussianBlur(blurred, (31, 31), 0)
    
    blurred = np.repeat(blurred[...,np.newaxis], 3, -1)
    
    sq3 = mirror[0:rois, 2*rois:fw]
    
    thresh = cv2.cvtColor(sq3, cv2.COLOR_BGR2GRAY)
    thresh = cv2.GaussianBlur(thresh, (21,21), 0)
    _, thresh = cv2.threshold(thresh, 127, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    thresh = np.repeat(thresh[...,np.newaxis], 3, -1) 
    
    sq4 = mirror[rois:2*rois, 2*rois:fw]
    
    canny = cv2.Canny(sq4, 50, 150)
    canny = np.repeat(canny[..., np.newaxis], 3, -1)
    
    
    
    mirror[0:rois, 0:rois] = grey 
    mirror[0:rois, rois:2*rois] = blurred
    mirror[0:rois, 2*rois:fw] = thresh
    mirror[rois:2*rois, 2*rois:fw] = canny
    
    cv2.imshow("Webcam", mirror)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
    
cap.release()              
cv2.destroyAllWindows() 